In [1]:
# Cargamos la librerías
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fiona
import rasterio.env
import rasterio
import rasterio.plot as rioplot
import rasterstats as rs
import gdal
import os
import shapely.geometry
from netCDF4 import Dataset


In [2]:
###Cargamos nuestro arquivo json

json_data=pd.read_json(r'D:\Python\GeoPandas\netCDF\json\0_10000\dataframeZonalnetCDF_3.json')

In [3]:
### Revisamos la dimensión en KB de cada json

# json completo
json_data.info(memory_usage='deep')

# json con las columnas necesarias
json_data[['LLAVE','zonal_mean','anio','mes','dia','hora','capa']].info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152000 entries, 0 to 151999
Data columns (total 16 columns):
CVE_ENT        152000 non-null int64
CVE_MUN        152000 non-null int64
LLAVE          152000 non-null int64
NOM_MUN        152000 non-null object
OID_1          152000 non-null int64
cov_           152000 non-null int64
cov_id         152000 non-null int64
zonal_min      152000 non-null float64
zonal_max      152000 non-null float64
zonal_mean     152000 non-null float64
zonal_count    152000 non-null int64
anio           152000 non-null int64
mes            152000 non-null int64
dia            152000 non-null int64
hora           152000 non-null int64
capa           152000 non-null int64
dtypes: float64(3), int64(12), object(1)
memory usage: 29.4 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 152000 entries, 0 to 151999
Data columns (total 7 columns):
LLAVE         152000 non-null int64
zonal_mean    152000 non-null float64
anio          152000 non-null int64
mes    

In [4]:
### Nos quedamos sólo con las columnas necesarias
json_data_min=json_data[['LLAVE','zonal_mean','anio','mes','dia','hora','capa']]
json_data_min

,LLAVE,zonal_mean,anio,mes,dia,hora,capa
0,15038,290.642258,2004,10,1,1,7
1,15046,290.962769,2004,10,1,1,7
2,15060,290.532247,2004,10,1,1,7
3,15095,290.146606,2004,10,1,1,7
4,15112,290.316986,2004,10,1,1,7
...,...,...,...,...,...,...,...
151995,9010,296.324341,2008,3,3,3,6
151996,9004,296.324341,2008,3,3,3,6
151997,9003,296.324341,2008,3,3,3,6
151998,9011,297.110855,2008,3,3,3,6


In [5]:
# Cambiamos el formato de las fechas para a dos dígitos (con el cero al inicio si sólo hay un dígito)
json_data_min['anio']=json_data_min['anio'].map("{:04}".format)
json_data_min['mes']=json_data_min['mes'].map("{:02}".format)
json_data_min['dia']=json_data_min['dia'].map("{:02}".format)
json_data_min

C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value

,LLAVE,zonal_mean,anio,mes,dia,hora,capa
0,15038,290.642258,2004,10,01,1,7
1,15046,290.962769,2004,10,01,1,7
2,15060,290.532247,2004,10,01,1,7
3,15095,290.146606,2004,10,01,1,7
4,15112,290.316986,2004,10,01,1,7
...,...,...,...,...,...,...,...
151995,9010,296.324341,2008,03,03,3,6
151996,9004,296.324341,2008,03,03,3,6
151997,9003,296.324341,2008,03,03,3,6
151998,9011,297.110855,2008,03,03,3,6


In [6]:
## Cambiamos el formato de las horas
json_data_min.loc[json_data_min['hora']==0,'hora']='06:00:00'
json_data_min.loc[json_data_min['hora']==1,'hora']='12:00:00'
json_data_min.loc[json_data_min['hora']==2,'hora']='18:00:00'
json_data_min.loc[json_data_min['hora']==3,'hora']='24:00:00'

json_data_min

C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,LLAVE,zonal_mean,anio,mes,dia,hora,capa
0,15038,290.642258,2004,10,01,12:00:00,7
1,15046,290.962769,2004,10,01,12:00:00,7
2,15060,290.532247,2004,10,01,12:00:00,7
3,15095,290.146606,2004,10,01,12:00:00,7
4,15112,290.316986,2004,10,01,12:00:00,7
...,...,...,...,...,...,...,...
151995,9010,296.324341,2008,03,03,24:00:00,6
151996,9004,296.324341,2008,03,03,24:00:00,6
151997,9003,296.324341,2008,03,03,24:00:00,6
151998,9011,297.110855,2008,03,03,24:00:00,6


In [7]:
### Generamos una columna que contendrá la fecha y la hora

json_data_min['date_time']=json_data_min['anio'].str.cat(json_data_min[['mes', 'dia']], sep='-')
json_data_min['date_time']=json_data_min['date_time'].str.cat(json_data_min[['hora']],sep=' ')
json_data_min

C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,LLAVE,zonal_mean,anio,mes,dia,hora,capa,date_time
0,15038,290.642258,2004,10,01,12:00:00,7,2004-10-01 12:00:00
1,15046,290.962769,2004,10,01,12:00:00,7,2004-10-01 12:00:00
2,15060,290.532247,2004,10,01,12:00:00,7,2004-10-01 12:00:00
3,15095,290.146606,2004,10,01,12:00:00,7,2004-10-01 12:00:00
4,15112,290.316986,2004,10,01,12:00:00,7,2004-10-01 12:00:00
...,...,...,...,...,...,...,...,...
151995,9010,296.324341,2008,03,03,24:00:00,6,2008-03-03 24:00:00
151996,9004,296.324341,2008,03,03,24:00:00,6,2008-03-03 24:00:00
151997,9003,296.324341,2008,03,03,24:00:00,6,2008-03-03 24:00:00
151998,9011,297.110855,2008,03,03,24:00:00,6,2008-03-03 24:00:00


In [8]:
### Nos quedamos sólo con cuatro columnas LLAVE, zonal_mean, date_time y capa
json_data_min[['LLAVE','zonal_mean','date_time','capa']]

,LLAVE,zonal_mean,date_time,capa
0,15038,290.642258,2004-10-01 12:00:00,7
1,15046,290.962769,2004-10-01 12:00:00,7
2,15060,290.532247,2004-10-01 12:00:00,7
3,15095,290.146606,2004-10-01 12:00:00,7
4,15112,290.316986,2004-10-01 12:00:00,7
...,...,...,...,...
151995,9010,296.324341,2008-03-03 24:00:00,6
151996,9004,296.324341,2008-03-03 24:00:00,6
151997,9003,296.324341,2008-03-03 24:00:00,6
151998,9011,297.110855,2008-03-03 24:00:00,6


### Haremos una función que realice los pasos anteriores

In [9]:
def json_convert_min_col(data):
    
    ### Nos quedamos sólo con las columnas necesarias
    data_min=data[['LLAVE','NOM_MUN','zonal_mean','anio','mes','dia','hora','capa']]
    
    # Cambiamos el formato de las fechas
    data_min['anio']=data_min['anio'].map("{:04}".format)
    data_min['mes']=data_min['mes'].map("{:02}".format)
    data_min['dia']=data_min['dia'].map("{:02}".format)
    
    # Cambiamos el formato de las horas
    data_min.loc[data_min['hora']==0,'hora']='06:00:00'
    data_min.loc[data_min['hora']==1,'hora']='12:00:00'
    data_min.loc[data_min['hora']==2,'hora']='18:00:00'
    data_min.loc[data_min['hora']==3,'hora']='24:00:00'
    
    ### Generamos una columna que contendrá la fecha y la hora
    data_min['date_time']=data_min['anio'].str.cat(data_min[['mes', 'dia']], sep='-')
    data_min['date_time']=data_min['date_time'].str.cat(data_min[['hora']],sep=' ')
    
    ### Nos quedamos sólo con cuatro columnas LLAVE, zonal_mean, date_time y capa
    data_min=data_min[['LLAVE','NOM_MUN','zonal_mean','date_time','capa']]
    
    return data_min

In [10]:
## Probamos la función
#json_convert_min_col(json_data).info(memory_usage='deep')
json_convert_min_col(json_data)

C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,LLAVE,NOM_MUN,zonal_mean,date_time,capa
0,15038,Isidro Fabela,290.642258,2004-10-01 12:00:00,7
1,15046,Jilotzingo,290.962769,2004-10-01 12:00:00,7
2,15060,Nicolás Romero,290.532247,2004-10-01 12:00:00,7
3,15095,Tepotzotlán,290.146606,2004-10-01 12:00:00,7
4,15112,Villa del Carbón,290.316986,2004-10-01 12:00:00,7
...,...,...,...,...,...
151995,9010,Álvaro Obregón,296.324341,2008-03-03 24:00:00,6
151996,9004,Cuajimalpa de Morelos,296.324341,2008-03-03 24:00:00,6
151997,9003,Coyoacán,296.324341,2008-03-03 24:00:00,6
151998,9011,Tláhuac,297.110855,2008-03-03 24:00:00,6


### Aplicaremos esta función los json que tenemos

In [11]:
## Generamos un data frame que hará el concentrado al ir agregando el resultado de cada json. 
data_frame_resumen=pd.DataFrame(columns=['LLAVE','NOM_MUN','zonal_mean','anio','mes','dia','hora','capa'])



In [12]:
import time
import requests
from tqdm import tqdm

direcciones=[r'D:\Python\GeoPandas\netCDF\json\0_10000',
            r'D:\Python\GeoPandas\netCDF\json\10000_20000',
            r'D:\Python\GeoPandas\netCDF\json\20000_30000',
            r'D:\Python\GeoPandas\netCDF\json\30000_36520']

start = time.time()




for i in direcciones:
    for j in tqdm(range(5)):
        datos=pd.read_json(i+'\dataframeZonalnetCDF_'+str(j)+".json")
        data_frame_resumen=data_frame_resumen.append(datos[['LLAVE','NOM_MUN','zonal_mean','anio','mes','dia','hora','capa']])

end = time.time()
print(end - start)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.05s/it]


87.45469403266907


In [13]:
## Aplicamos la función al data frame
data_frame_resumen_min=json_convert_min_col(data_frame_resumen)
## Ordenamos el data frame
data_frame_resumen_min=data_frame_resumen_min.sort_values(by=['LLAVE','date_time'])
# Convertimos el dato de temperatura media a flotante 
data_frame_resumen_min['zonal_mean']=data_frame_resumen_min['zonal_mean'].astype('float64')

In [14]:
## Aquí guardamos el data frame de las capas 6 y 7 en json para que después lo agreguemos al resumen
## se hizo este mismo procedimiento a las capas 2-3 y 4-5
data_frame_resumen_min.reset_index().to_json(r'D:\Python\GeoPandas\netCDF\json\capas_6_7.json')

In [16]:
## Aquí hacemos un reshape del formato del data frame. Como veremos en la siguiente tabla, la columna "capa" contiene el valor
## de cada capa
data_frame_resumen_min

,LLAVE,NOM_MUN,zonal_mean,date_time,capa
5,9002,Azcapotzalco,294.453339,2004-10-01 06:00:00,6
5,9002,Azcapotzalco,292.574341,2004-10-01 06:00:00,7
5,9002,Azcapotzalco,292.359833,2004-10-01 12:00:00,6
5,9002,Azcapotzalco,290.962769,2004-10-01 12:00:00,7
81,9002,Azcapotzalco,289.287415,2004-10-01 18:00:00,7
...,...,...,...,...,...
99016,15125,Tonanitla,292.478363,2017-03-31 12:00:00,6
99092,15125,Tonanitla,292.574921,2017-03-31 18:00:00,6
99092,15125,Tonanitla,291.678375,2017-03-31 18:00:00,7
99092,15125,Tonanitla,291.692291,2017-03-31 24:00:00,6


In [17]:
## La idea es convertir esas capas a columnas, para poder hacer la resta más sencilla.
## Esto lo hacemos con la funcion .pivot_table de pandas. Tomamos como índices  a LLAVE, NOM_MUN y date_time
## Las columnas serán las categorías de capa
## Los valores serán los de las temperaturas medias
data_frame_resumen_min_pivot=data_frame_resumen_min.pivot_table(index=['LLAVE','NOM_MUN','date_time'] ,columns='capa',values='zonal_mean')

In [18]:
data_frame_resumen_min_pivot


capa                                             6           7
LLAVE NOM_MUN      date_time                                  
9002  Azcapotzalco 2004-10-01 06:00:00  294.453339  292.574341
                   2004-10-01 12:00:00  292.359833  290.962769
                   2004-10-01 18:00:00  290.609863  289.287415
                   2004-10-01 24:00:00  294.125610  291.618378
                   2004-10-02 06:00:00  294.835327  292.938416
...                                            ...         ...
15125 Tonanitla    2017-03-30 24:00:00  293.926575  291.954803
                   2017-03-31 06:00:00  290.185791  289.554779
                   2017-03-31 12:00:00  292.478363  292.498810
                   2017-03-31 18:00:00  292.574921  291.678375
                   2017-03-31 24:00:00  291.692291  289.586151

[1387760 rows x 2 columns]

### Incorporamos los data frame con las distintas capas 

In [19]:
# Cargamos los data frame
json_data_2_3=pd.read_json(r'D:\Python\GeoPandas\netCDF\json\capas_2_3.json')
json_data_4_5=pd.read_json(r'D:\Python\GeoPandas\netCDF\json\capas_4_5.json')
json_data_6_7=pd.read_json(r'D:\Python\GeoPandas\netCDF\json\capas_6_7.json')

In [20]:
# Creamos un data frame que contendrá la información de los df cargados.
# Nos quedaremos sólo con las columnas 'LLAVE','NOM_MUN','zonal_mean','date_time','capa'
json_resumen=pd.DataFrame(columns=['LLAVE','NOM_MUN','zonal_mean','date_time','capa'])

# Agregamos los data frame al data frame resumen

json_resumen=json_resumen.append(json_data_2_3[['LLAVE','NOM_MUN','zonal_mean','date_time','capa']])
json_resumen=json_resumen.append(json_data_4_5[['LLAVE','NOM_MUN','zonal_mean','date_time','capa']])
json_resumen=json_resumen.append(json_data_6_7[['LLAVE','NOM_MUN','zonal_mean','date_time','capa']])

In [21]:
## Como en la operación 17, hacemos el reshape del data frame convirtiendo en columnas cada categoría de la columna capa
json_resumen_pivot=json_resumen.pivot_table(index=['LLAVE','NOM_MUN','date_time'] ,columns='capa',values='zonal_mean')
json_resumen_pivot

capa                                               2             3  \
LLAVE NOM_MUN      date_time                                         
9002  Azcapotzalco 2004-10-01 06:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 12:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 18:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 24:00:00  1.000000e+15  1.000000e+15   
                   2004-10-02 06:00:00  1.000000e+15  1.000000e+15   
...                                              ...           ...   
15125 Tonanitla    2017-03-30 24:00:00  1.000000e+15  3.008447e+02   
                   2017-03-31 06:00:00  1.000000e+15  2.963118e+02   
                   2017-03-31 12:00:00  1.000000e+15  2.931535e+02   
                   2017-03-31 18:00:00  1.000000e+15  2.941316e+02   
                   2017-03-31 24:00:00  1.000000e+15  2.992180e+02   

capa                                               4           5           6  \
LLAVE NOM_MUN      date_time                                                   
9002  Azcapotzalco 2004-10-01 06:00:00  1.000000e+15  296.532867  294.453339   
                   2004-10-01 12:00:00  1.000000e+15  293.255554  292.359833   
                   2004-10-01 18:00:00  1.000000e+15  290.729401  290.609863   
                   2004-10-01 24:00:00  1.000000e+15  296.978516  294.125610   
                   2004-10-02 06:00:00  1.000000e+15  296.816742  294.835327   
...                                              ...         ...         ...   
15125 Tonanitla    2017-03-30 24:00:00  2.982184e+02  295.980957  293.926575   
                   2017-03-31 06:00:00  2.938672e+02  291.665161  290.185791   
                   2017-03-31 12:00:00  2.926771e+02  292.020905  292.478363   
                   2017-03-31 18:00:00  2.936372e+02  293.221832  292.574921   
                   2017-03-31 24:00:00  2.965123e+02  294.015839  291.692291   

capa                                             7  
LLAVE NOM_MUN      date_time                        
9002  Azcapotzalco 2004-10-01 06:00:00  292.574341  
                   2004-10-01 12:00:00  290.962769  
                   2004-10-01 18:00:00  289.287415  
                   2004-10-01 24:00:00  291.618378  
                   2004-10-02 06:00:00  292.938416  
...                                            ...  
15125 Tonanitla    2017-03-30 24:00:00  291.954803  
                   2017-03-31 06:00:00  289.554779  
                   2017-03-31 12:00:00  292.498810  
                   2017-03-31 18:00:00  291.678375  
                   2017-03-31 24:00:00  289.586151  

[1387760 rows x 6 columns]

In [22]:
# Hacemos las respectivas restas
json_resumen_pivot['3-2']=json_resumen_pivot[3]-json_resumen_pivot[2]
json_resumen_pivot['4-3']=json_resumen_pivot[4]-json_resumen_pivot[3]
json_resumen_pivot['5-4']=json_resumen_pivot[5]-json_resumen_pivot[4]
json_resumen_pivot['6-5']=json_resumen_pivot[6]-json_resumen_pivot[5]
json_resumen_pivot['7-6']=json_resumen_pivot[7]-json_resumen_pivot[6]
json_resumen_pivot

capa                                               2             3  \
LLAVE NOM_MUN      date_time                                         
9002  Azcapotzalco 2004-10-01 06:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 12:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 18:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 24:00:00  1.000000e+15  1.000000e+15   
                   2004-10-02 06:00:00  1.000000e+15  1.000000e+15   
...                                              ...           ...   
15125 Tonanitla    2017-03-30 24:00:00  1.000000e+15  3.008447e+02   
                   2017-03-31 06:00:00  1.000000e+15  2.963118e+02   
                   2017-03-31 12:00:00  1.000000e+15  2.931535e+02   
                   2017-03-31 18:00:00  1.000000e+15  2.941316e+02   
                   2017-03-31 24:00:00  1.000000e+15  2.992180e+02   

capa                                               4           5           6  \
LLAVE NOM_MUN      date_time                                                   
9002  Azcapotzalco 2004-10-01 06:00:00  1.000000e+15  296.532867  294.453339   
                   2004-10-01 12:00:00  1.000000e+15  293.255554  292.359833   
                   2004-10-01 18:00:00  1.000000e+15  290.729401  290.609863   
                   2004-10-01 24:00:00  1.000000e+15  296.978516  294.125610   
                   2004-10-02 06:00:00  1.000000e+15  296.816742  294.835327   
...                                              ...         ...         ...   
15125 Tonanitla    2017-03-30 24:00:00  2.982184e+02  295.980957  293.926575   
                   2017-03-31 06:00:00  2.938672e+02  291.665161  290.185791   
                   2017-03-31 12:00:00  2.926771e+02  292.020905  292.478363   
                   2017-03-31 18:00:00  2.936372e+02  293.221832  292.574921   
                   2017-03-31 24:00:00  2.965123e+02  294.015839  291.692291   

capa                                             7           3-2       4-3  \
LLAVE NOM_MUN      date_time                                                 
9002  Azcapotzalco 2004-10-01 06:00:00  292.574341  0.000000e+00  0.000000   
                   2004-10-01 12:00:00  290.962769  0.000000e+00  0.000000   
                   2004-10-01 18:00:00  289.287415  0.000000e+00  0.000000   
                   2004-10-01 24:00:00  291.618378  0.000000e+00  0.000000   
                   2004-10-02 06:00:00  292.938416  0.000000e+00  0.000000   
...                                            ...           ...       ...   
15125 Tonanitla    2017-03-30 24:00:00  291.954803 -1.000000e+15 -2.626373   
                   2017-03-31 06:00:00  289.554779 -1.000000e+15 -2.444611   
                   2017-03-31 12:00:00  292.498810 -1.000000e+15 -0.476440   
                   2017-03-31 18:00:00  291.678375 -1.000000e+15 -0.494385   
                   2017-03-31 24:00:00  289.586151 -1.000000e+15 -2.705719   

capa                                             5-4       6-5       7-6  
LLAVE NOM_MUN      date_time                                              
9002  Azcapotzalco 2004-10-01 06:00:00 -1.000000e+15 -2.079529 -1.878998  
                   2004-10-01 12:00:00 -1.000000e+15 -0.895721 -1.397064  
                   2004-10-01 18:00:00 -1.000000e+15 -0.119537 -1.322449  
                   2004-10-01 24:00:00 -1.000000e+15 -2.852905 -2.507233  
                   2004-10-02 06:00:00 -1.000000e+15 -1.981415 -1.896912  
...                                              ...       ...       ...  
15125 Tonanitla    2017-03-30 24:00:00 -2.237396e+00 -2.054382 -1.971771  
                   2017-03-31 06:00:00 -2.202026e+00 -1.479370 -0.631012  
                   2017-03-31 12:00:00 -6.561584e-01  0.457458  0.020447  
                   2017-03-31 18:00:00 -4.153748e-01 -0.646912 -0.896545  
                   2017-03-31 24:00:00 -2.496460e+00 -2.323547 -2.106140  

[1387760 rows x 11 columns]

In [24]:
## Si el valor es negativo, truncamos a cero. En caso contrario, no hacemos nada

json_resumen_pivot.loc[json_resumen_pivot['3-2']<0,'3-2']=0
json_resumen_pivot.loc[json_resumen_pivot['4-3']<0,'4-3']=0
json_resumen_pivot.loc[json_resumen_pivot['5-4']<0,'5-4']=0
json_resumen_pivot.loc[json_resumen_pivot['6-5']<0,'6-5']=0
json_resumen_pivot.loc[json_resumen_pivot['7-6']<0,'7-6']=0

json_resumen_pivot

capa                                               2             3  \
LLAVE NOM_MUN      date_time                                         
9002  Azcapotzalco 2004-10-01 06:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 12:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 18:00:00  1.000000e+15  1.000000e+15   
                   2004-10-01 24:00:00  1.000000e+15  1.000000e+15   
                   2004-10-02 06:00:00  1.000000e+15  1.000000e+15   
...                                              ...           ...   
15125 Tonanitla    2017-03-30 24:00:00  1.000000e+15  3.008447e+02   
                   2017-03-31 06:00:00  1.000000e+15  2.963118e+02   
                   2017-03-31 12:00:00  1.000000e+15  2.931535e+02   
                   2017-03-31 18:00:00  1.000000e+15  2.941316e+02   
                   2017-03-31 24:00:00  1.000000e+15  2.992180e+02   

capa                                               4           5           6  \
LLAVE NOM_MUN      date_time                                                   
9002  Azcapotzalco 2004-10-01 06:00:00  1.000000e+15  296.532867  294.453339   
                   2004-10-01 12:00:00  1.000000e+15  293.255554  292.359833   
                   2004-10-01 18:00:00  1.000000e+15  290.729401  290.609863   
                   2004-10-01 24:00:00  1.000000e+15  296.978516  294.125610   
                   2004-10-02 06:00:00  1.000000e+15  296.816742  294.835327   
...                                              ...         ...         ...   
15125 Tonanitla    2017-03-30 24:00:00  2.982184e+02  295.980957  293.926575   
                   2017-03-31 06:00:00  2.938672e+02  291.665161  290.185791   
                   2017-03-31 12:00:00  2.926771e+02  292.020905  292.478363   
                   2017-03-31 18:00:00  2.936372e+02  293.221832  292.574921   
                   2017-03-31 24:00:00  2.965123e+02  294.015839  291.692291   

capa                                             7  3-2  4-3  5-4       6-5  \
LLAVE NOM_MUN      date_time                                                  
9002  Azcapotzalco 2004-10-01 06:00:00  292.574341  0.0  0.0  0.0  0.000000   
                   2004-10-01 12:00:00  290.962769  0.0  0.0  0.0  0.000000   
                   2004-10-01 18:00:00  289.287415  0.0  0.0  0.0  0.000000   
                   2004-10-01 24:00:00  291.618378  0.0  0.0  0.0  0.000000   
                   2004-10-02 06:00:00  292.938416  0.0  0.0  0.0  0.000000   
...                                            ...  ...  ...  ...       ...   
15125 Tonanitla    2017-03-30 24:00:00  291.954803  0.0  0.0  0.0  0.000000   
                   2017-03-31 06:00:00  289.554779  0.0  0.0  0.0  0.000000   
                   2017-03-31 12:00:00  292.498810  0.0  0.0  0.0  0.457458   
                   2017-03-31 18:00:00  291.678375  0.0  0.0  0.0  0.000000   
                   2017-03-31 24:00:00  289.586151  0.0  0.0  0.0  0.000000   

capa                                         7-6  
LLAVE NOM_MUN      date_time                      
9002  Azcapotzalco 2004-10-01 06:00:00  0.000000  
                   2004-10-01 12:00:00  0.000000  
                   2004-10-01 18:00:00  0.000000  
                   2004-10-01 24:00:00  0.000000  
                   2004-10-02 06:00:00  0.000000  
...                                          ...  
15125 Tonanitla    2017-03-30 24:00:00  0.000000  
                   2017-03-31 06:00:00  0.000000  
                   2017-03-31 12:00:00  0.020447  
                   2017-03-31 18:00:00  0.000000  
                   2017-03-31 24:00:00  0.000000  

[1387760 rows x 11 columns]

In [25]:
### Guardamos en formato dta
json_resumen_pivot.to_stata(r'D:\capas_2_7.dta')

C:\Users\miloc\AppData\Local\Continuum\miniconda3\envs\GDAL\lib\site-packages\pandas\io\stata.py:2303: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    2   ->   _2
    3   ->   _3
    4   ->   _4
    5   ->   _5
    6   ->   _6
    7   ->   _7
    b'3-2'   ->   _3_2
    b'4-3'   ->   _4_3
    b'5-4'   ->   _5_4
    b'6-5'   ->   _6_5
    b'7-6'   ->   _7_6

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)
